# PIPLEINE VERSIÓN 3 (PLS)

INSTALACIÓN E IMPORTACIÓN DE LIBRERÍAS

In [ ]:
# pip install numpy
# pip install pandas
# pip install scipy
# pip install lightgbm
# pip install dtw
# pip install statsmodels
# pip install sklearn
# pip install optuna

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import dtw
import matplotlib.pyplot as plt
import optuna

from scipy.stats import zscore
from statsmodels.tsa.stattools import acf, pacf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.manifold import MDS
from sklearn.cross_decomposition import PLSRegression


## 0. PRETRATAMIENTO GENÉRICO

In [ ]:
def pre_treatment(path = None, df = None, date_col_hint = None):
    """
    Carga datos desde cualquier CSV/Excel o acepta DataFrame genérico.
    Genera reporte de estructura, estadísticos, missing, outliers, distribuciones y boxplots.
    Devuelve DataFrame procesado listo para renombrar a df_0.
    """
    # Carga
    if df is None:
        if path.lower().endswith('.csv'):
            df = pd.read_csv(path)
        else:
            df = pd.read_excel(path)
    # Reporte inicial
    print('Shape:', df.shape)
    print('\nColumnas y tipos:\n', df.dtypes)
    print('\nPrimeras filas:\n', df.head())
    print('\nDescriptivos numéricos:\n', df.describe().T)
    # Valores faltantes
    miss = df.isnull().sum()
    print('\nValores faltantes por columna:\n', miss[miss > 0])
    # Distribuciones y boxplots
    num_cols = df.select_dtypes(include = [np.number]).columns
    if len(num_cols):
        df[num_cols].hist(figsize = (15, 10), bins = 30)
        plt.suptitle('Histogramas de variables numéricas'); plt.show()
        for col in num_cols:
            plt.figure()
            df.boxplot(column = col)
            plt.title(f'Boxplot {col}')
            plt.show()
    # Detección de outliers (z-score)
    outliers = {}
    for col in num_cols:
        zs = zscore(df[col].dropna())
        count = np.sum(np.abs(zs) > 3)
        if count > 0:
            outliers[col] = int(count)
    print('\nOutliers detectados (|z|>3) por columna:\n', outliers)
    # Inferencia de columnas de fecha y valor
    if date_col_hint and date_col_hint in df.columns:
        date_col = date_col_hint
    else:
        date_col = df.select_dtypes(include=['datetime', 'datetime64']).columns.tolist()
        date_col = date_col[0] if date_col else df.columns[0]
    value_cols = [c for c in df.columns if c not in [date_col] and df[c].dtype in [np.int64, np.float64]]
    # Formato long si wide
    if 'product_code' not in df.columns and len(value_cols) > 1:
        df_long = df.melt(id_vars = [date_col], var_name = 'product_code', value_name = 'demand')
    else:
        df_long = df.rename(columns = {date_col: 'date'})
        df_long = df_long.rename(columns = {value_cols[0]: 'demand'}) if 'product_code' in df.columns else df_long
    df_long['date'] = pd.to_datetime(df_long['date'])
    return df_long

## 1. CARGA Y LIMPIEZA DE DATOS

In [ ]:
def load_and_clean_data(path = None, df = None, date_col = 'date', value_col = 'demand'):
    """
    Carga datos desde CSV/Excel/SQL o acepta DataFrame.
    Limpia: detecta NaNs, outliers (zscore > 3) e imputa.
    Devuelve DataFrame long con columnas [product_code, date, demand].
    """
    if df is None:
        if path.endswith('.csv'):
            df = pd.read_csv(path)
        else:
            df = pd.read_excel(path)
    df = df.copy()
    if 'product_code' not in df.columns:
        df = df.melt(id_vars = [date_col], var_name = 'product_code', value_name = value_col)
    df[date_col] = pd.to_datetime(df[date_col])
    df[value_col] = df[value_col].fillna(0)
    def cap_outliers(x):
        zs = zscore(x)
        x[(zs > 3) | (zs < -3)] = x.median()
        return x
    df[value_col] = df.groupby('product_code')[value_col].transform(cap_outliers)
    return df[['product_code', date_col, value_col]]


## 2. ANÁLISIS EXPLORATORIO

In [ ]:
def exploratory_analysis(df_long, date_col = 'date', value_col = 'demand', n_top = 5):
    """
    Estadísticas básicas, % ceros, ACF/PACF global y por serie.
    Genera gráficos de distribuciones, ACF/PACF y ejemplos de series.
    """
    stats = df_long.groupby('product_code')[value_col].agg(
        mean = 'mean', var = 'var', pct_zero = lambda x: (x == 0).mean()).reset_index()
    print(stats.head(n_top))

    # Histograma de medias
    plt.figure()
    stats['mean'].hist(bins = 30)
    plt.title('Distribución de medias por SKU')
    plt.show()

    # ACF/PACF global
    y = df_long.groupby(date_col)[value_col].sum().sort_index()
    acf_vals = acf(y, nlags = 12); pacf_vals = pacf(y, nlags = 12)
    plt.figure()
    plt.stem(acf_vals)
    plt.title('ACF Global')
    plt.show()

    plt.figure()
    plt.stem(pacf_vals)
    plt.title('PACF Global')
    plt.show()
    
    # Muestra algunas series individuales
    sample_skus = df_long['product_code'].unique()[:n_top]
    plt.figure()
    for sku in sample_skus:
        sub = df_long[df_long['product_code'] == sku]
        plt.plot(sub[date_col], sub[value_col], label = str(sku), alpha = 0.7)
    plt.title('Series de muestra')
    plt.legend()
    plt.show()

## 3. CLUSTERING DE SERIES

In [ ]:
def cluster_series(df_long, k_range = range(2, 8), random_state = 42):
    """
    Calcula matriz DTW entre series, selecciona k óptimo (silhouette/elbow).
    Retorna dict sku->cluster, medoids y MDS del embedding.
    """
    pivot = df_long.pivot(index = 'product_code', columns = 'date', values = 'demand').fillna(0)
    series_list = pivot.values; N = len(series_list)
    D = np.zeros((N, N))
    for i in range(N):
        for j in range(i + 1, N):
            d = dtw.distance(series_list[i], series_list[j])
            D[i,j] = D[j,i] = d
    from sklearn_extra.cluster import KMedoids
    best_k = 4  # o buscar en k_range con silhouette_score
    km = KMedoids(n_clusters = best_k, metric = 'precomputed', random_state = random_state)
    labels = km.fit_predict(D)
    mapping = dict(zip(pivot.index, labels))
    
    # Embedding 2D
    mds = MDS(n_components = 2, dissimilarity = 'precomputed', random_state = random_state)
    coords = mds.fit_transform(D)
    plt.figure()
    plt.scatter(coords[:, 0], coords[:, 1], c = labels, cmap = 'tab10', alpha = 0.7)
    plt.title('MDS de series por cluster')
    plt.show()
    
    # Medoids
    meds = pivot.index[km.medoid_indices_]
    plt.figure()
    for m in meds:
        plt.plot(pivot.columns, pivot.loc[m], alpha = 0.8)
    plt.title('Medoids por cluster')
    plt.show()
    return mapping

## 4. SELECCIÓN DE LAGS POR CLUSTER

In [ ]:
def select_lags_per_cluster(df_long, mapping, date_col = 'date', value_col = 'demand', acf_thresh = 0.1):
    """
    Para cada cluster agrupa series, calcula ACF/PACF medias,
    selecciona lags donde |ACF|>=acf_thresh o test de Ljung-Box.
    Devuelve dict cluster->lista de lags.
    """
    df_long_ = df_long.copy()
    df_long_['cluster'] = df_long_['product_code'].map(mapping)
    lags_by_cluster = {}
    for cl in sorted(df_long_['cluster'].unique()):
        sub = df_long_[df_long_['cluster'] == cl]
        pivot = sub.pivot(index = date_col, columns = 'product_code', values = value_col).fillna(0)
        acf_vals = acf(pivot.mean(axis = 1), nlags = 24)
        pacf_vals = pacf(pivot.mean(axis = 1), nlags = 24)
        lags = [i for i, v in enumerate(acf_vals) if abs(v) >= acf_thresh]
        
        # podríamos agregar condición de significancia: prueba Ljung-Box
        lags_by_cluster[cl] = sorted(set(lags))
        print(f"Cluster {cl}: lags seleccionados -> {lags_by_cluster[cl]}")
    return lags_by_cluster

## 5. GENERACIÓN DE FEATURES DINÁMICAS

In [ ]:
def make_features(df_long, mapping, lags_by_cluster, date_col = 'date', value_col = 'demand', h = 3):
    """
    Recorre cada SKU, aplica lags dinámicos según cluster,
    genera stats móviles, Croston y calendariales.
    """
    features = {}
    for sku, cl in mapping.items():
        series = df_long[df_long['product_code'] == sku].set_index(date_col)[value_col]
        lags = lags_by_cluster[cl]
        df = pd.DataFrame({'y': series})
        for lag in lags:
            df[f'lag_{lag}'] = df['y'].shift(lag)
            df[f'roll_mean_{lag}'] = df['y'].shift(1).rolling(window = lag, min_periods = 1).mean()
        
        # Croston (como antes)
        df = df.reset_index()
        df['month'] = df[date_col].dt.month
        df.dropna(inplace = True)
        df['y_bin'] = (df['y'] > 0).astype(int)
        df['y_reg'] = df['y']
        features[sku] = (df.iloc[:-h], df.iloc[-h:])
    return features

## 6. APLICACIÓN GLOBAL DE PLS

In [ ]:
def apply_global_pls(features_dict, n_components = 2):
    """
    Ajusta un modelo PLS sobre el conjunto de entrenamiento agregado de todas las series,
    transforma cada train/test en componentes latentes y los añade como nuevas columnas.
    Retorno un nuevo dict sku -> (train_df_pls, test_df_pls).
    """
    # Concatenar todos los train
    all_tr = []
    for sku, (tr, _) in features_dict.items():
        df = tr.copy()
        df['sku'] = tr.copy()
        all_tr.append(df)
    big_tr = pd.concat(all_tr, ignore_index = True)
    X = big_tr.drop(['date', 'y', 'y_bin', 'y_reg', 'sku'], axis = 1).values
    y = big_tr['y_reg'].values
    scaler = MinMaxScaler()
    Xs = scaler.fit_transform(X)
    pls = PLSRegression(n_components = n_components)
    pls.fit(Xs, y)

    # Transformar cada train/test
    new_dict = {}
    for sku, (tr, te) in features_dict.items():
        for df_ in [tr, te]:
            X_df = df_.drop(['date', 'y', 'y_bin', 'y_reg'], axis = 1).values
            Xs_df = scaler.transform(X_df)
            comps = pls.transform(Xs_df)
            for i in range(n_components):
                df_[f'pls_comp_{i+1}'] = comps[:, i]
        new_dict[sku] = (tr, te)
    return new_dict

# def pls_feature_analysis(train_df, n_components=2):
#     """
#     Reduce dimensiones con PLS: identifica combinaciones lineales de features
#     que maximizan la covarianza con la variable objetivo.
#     Grafica los primeros componentes.
#     """
#     X = train_df.drop(['date','y','y_bin','y_reg'], axis=1).values
#     y = train_df['y_reg'].values
#     scaler = MinMaxScaler()
#     Xs = scaler.fit_transform(X)
#     pls = PLSRegression(n_components=n_components)
#     comps = pls.fit_transform(Xs, y)[0]
#     plt.figure(); plt.scatter(comps[:,0], comps[:,1], c=y, cmap='viridis', alpha=0.7)
#     plt.title('PLS Feature Space'); plt.xlabel('Comp1'); plt.ylabel('Comp2'); plt.colorbar(label='y'); plt.show()
#     return pls

## 7. MODELOS LIGHTGBM

In [ ]:
def train_pure_lgbm(train_df, params = None):
    X = train_df.drop(['date', 'y', 'y_bin', 'y_reg'], axis = 1)
    y = train_df['y_reg']
    m = lgb.LGBMRegressor(**(params or {}))
    m.fit(X, y)
    return m

def train_two_stage_lgbm(train_df, clf_params, reg_params):
    X = train_df.drop(['date', 'y', 'y_bin', 'y_reg'], axis = 1)
    clf = lgb.LGBMClassifier(**clf_params)
    clf.fit(X, train_df['y_bin'])
    reg = lgb.LGBMRegressor(**reg_params)
    pos = train_df['y_bin'] == 1
    reg.fit(X[pos], train_df.loc[pos,'y_reg'])
    return clf, reg

## 8. BACKTEST

In [ ]:
def backtest(df_long, mapping, features, model_wrapper, params, h = 3):
    recs = []
    for sku, (tr, te) in features.items():
        mdl = model_wrapper(tr, params)
        X_te = te.drop(['date', 'y', 'y_bin', 'y_reg'], axis = 1)
        if isinstance(mdl, tuple):
            p = mdl[0].predict(X_te) * mdl[1].predict(X_te)
        else:
            p = mdl.predict(X_te)
        recs.append({'sku': sku,
                     'mae': mean_absolute_error(te['y_reg'], p),
                     'rmse': np.sqrt(mean_squared_error(te['y_reg'], p)),
                     'mape': np.mean(np.abs((te['y_reg'] - p) / (te['y_reg'] + 1e-6)))})
    return pd.DataFrame(recs)

## 9. ANÁLISIS PLS DE FEATURES (REVISIÓN)

In [ ]:
def pls_feature_analysis(train_df, n_components = 2):
    """
    Reduce dimensiones con PLS: identifica combinaciones lineales de features
    que maximizan la covarianza con la variable objetivo. (Inspecciona covarianza con target).
    Grafica Scree-Plot y biplot por componentes.
    Grafica los primeros componentes.
    """
    X = train_df.drop(['date', 'y', 'y_bin', 'y_reg'], axis = 1).values
    y = train_df['y_reg'].values
    scaler = MinMaxScaler()
    Xs = scaler.fit_transform(X)
    pls = PLSRegression(n_components = n_components)
    Xs_scores, _ = pls.fit_transform(Xs, y)

    # Scree-Plot de Varianza Explicada
    var_x = np.var(Xs_scores, axis = 0)
    var_y = np.var(pls.y_scores_, axis = 0)
    cum_x = np.cumsum(var_x) / np.sum(var_x)
    cum_y = np.cumsum(var_y) / np.sum(var_y)
    plt.figure()
    plt.plot(range(1, len(cum_x) + 1), cum_x, marker = 'o', label = 'X')
    plt.plot(range(1, len(cum_y) + 1), cum_y, marker = 'o', label = 'Y')
    plt.title('Scree-Plot PLS')
    plt.xlabel('Componente')
    plt.ylabel('Varianza Explicada Acumulada')
    plt.legend()
    plt.show()

    # Biplot
    plt.figure()
    plt.scatter(Xs_scores[:, 0], Xs_scores[:, 1], c = y, cmap = 'viridis', alpha = 0.7)
    loadings = pls.x_loadings_
    for i, feat in enumerate(train_df.drop(['date', 'y', 'y_bin', 'y_reg'], axis = 1).columns):
        plt.arrow(0, 0, loadings[i, 0] * max(Xs_scores[:, 0]), loadings[i, 1] * max(Xs_scores[:, 1]), color = 'r', head_widht = 0.02)
        plt.text(loadings[i, 0] * max(Xs_scores[:, 0]) * 1.1, loadings[i, 1] * max(Xs_scores[:, 1]) * 1.1, feat, fontsize = 8)
    plt.title('Biplot PLS')
    plt.xlabel('Comp1')
    plt.ylabel('Comp2')
    plt.colorbar(label = 'y')
    plt.show()
    return pls



## 10. EJECUCIÓN END-TO-END

In [ ]:
if __name__ == '__main__':
    df_long = load_and_clean_data(path = 'df_0_long.xlsx')
    exploratory_analysis(df_long)
    mapping = cluster_series(df_long)
    lags_by_cluster = select_lags_per_cluster(df_long, mapping)
    features = make_features(df_long, mapping, lags_by_cluster)
    # 1. PLS supervisado global -> añade pls_comp_{i}
    features_pls = apply_global_pls(features, n_components = 3)
    # 2. Benchmark LightGBM puro
    pure_pls = backtest(df_long, mapping, features_pls, train_pure_lgbm, {'learning_rate': 0.05, 'num_leaves': 31})
    print('Puro + PLS: ', pure_pls.describe())
    # 3. Benchmark Two-Stage
    two_stage_pls = backtest(df_long, mapping, features_pls, lambda df, _: train_two_stage_lgbm(df, clf_params, reg_params), None)
    print('Two-Stage + PLS:', two_stage_pls.describe())